<a href="https://colab.research.google.com/github/peteray-dev/Basic-EDA/blob/master/Topic_pred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import userdata
import os

os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')

In [ ]:
!kaggle datasets download -d barclaysav/b-interview-arxiv-dataset


Dataset URL: https://www.kaggle.com/datasets/barclaysav/b-interview-arxiv-dataset
License(s): unknown
  0% 0.00/21.4M [00:00<?, ?B/s]
100% 21.4M/21.4M [00:00<00:00, 762MB/s]


In [ ]:

! mkdir dataset

! unzip /content/b-interview-arxiv-dataset.zip -d dataset

Archive:  /content/b-interview-arxiv-dataset.zip
  inflating: dataset/interview_data/arxiv_data.csv  
  inflating: dataset/interview_data/category_taxonomy.csv  


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MultiLabelBinarizer

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.model_selection import train_test_split


In [ ]:
paper_df = pd.read_csv('/content/dataset/interview_data/arxiv_data.csv')
category = pd.read_csv('/content/dataset/interview_data/category_taxonomy.csv')

In [ ]:
paper_df.head()

,titles,summaries,terms
0,Survey on Semantic Stereo Matching / Semantic ...,Stereo matching is one of the widely used tech...,"['cs.CV', 'cs.LG']"
1,FUTURE-AI: Guiding Principles and Consensus Re...,The recent advancements in artificial intellig...,"['cs.CV', 'cs.AI', 'cs.LG']"
2,Enforcing Mutual Consistency of Hard Regions f...,"In this paper, we proposed a novel mutual cons...","['cs.CV', 'cs.AI']"
3,Parameter Decoupling Strategy for Semi-supervi...,Consistency training has proven to be an advan...,['cs.CV']
4,Background-Foreground Segmentation for Interio...,"To ensure safety in automated driving, the cor...","['cs.CV', 'cs.LG']"


In [ ]:
category.head()

,category,code,name
0,Computer Science,cs.AI,Artificial Intelligence
1,Computer Science,cs.AR,Hardware Architecture
2,Computer Science,cs.CC,Computational Complexity
3,Computer Science,cs.CE,"Computational Engineering, Finance, and Science"
4,Computer Science,cs.CG,Computational Geometry


In [ ]:
paper_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51774 entries, 0 to 51773
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   titles     51774 non-null  object
 1   summaries  51774 non-null  object
 2   terms      51774 non-null  object
dtypes: object(3)
memory usage: 1.2+ MB


In [ ]:
paper_df.isnull().sum()

,0
titles,0
summaries,0
terms,0


In [ ]:
# sns.displot(paper_df['terms'], kde=True)


In [ ]:
paper_df['terms'].value_counts()


,count
terms,
['cs.CV'],17775
"['cs.LG', 'stat.ML']",5439
['cs.LG'],2832
"['cs.CV', 'cs.LG']",2145
"['cs.LG', 'cs.AI']",1809
...,...
"['cs.LG', 'cs.CV', 'physics.chem-ph']",1
"['cs.LG', 'cond-mat.dis-nn', 'cond-mat.mtrl-sci', 'physics.data-an']",1
"['stat.ML', 'cs.LG', 'physics.app-ph']",1


Data Cleaning

In [ ]:
def clean_and_split_terms(df, column='terms'):
    """
    Clean the 'terms' column by removing brackets, quotes, and spaces,
    then convert it into a list of topic codes.
    """
    import re

    df[column] = (
        df[column]
        .str.replace(r"[\[\]' ]", '', regex=True)   # Remove [, ], ', and spaces
        .str.split(',')                             # Split into list
    )
    return df
paper_df = clean_and_split_terms(paper_df)

In [ ]:
paper_df.head()

# the titles can be combined with the summaries




,titles,summaries,terms
0,Survey on Semantic Stereo Matching / Semantic ...,Stereo matching is one of the widely used tech...,"[cs.CV, cs.LG]"
1,FUTURE-AI: Guiding Principles and Consensus Re...,The recent advancements in artificial intellig...,"[cs.CV, cs.AI, cs.LG]"
2,Enforcing Mutual Consistency of Hard Regions f...,"In this paper, we proposed a novel mutual cons...","[cs.CV, cs.AI]"
3,Parameter Decoupling Strategy for Semi-supervi...,Consistency training has proven to be an advan...,[cs.CV]
4,Background-Foreground Segmentation for Interio...,"To ensure safety in automated driving, the cor...","[cs.CV, cs.LG]"


In [ ]:
# convert the term to binanry format

mlb = MultiLabelBinarizer()

y_bin = pd.DataFrame(mlb.fit_transform(paper_df['terms']),
                   columns=mlb.classes_,
                   index=paper_df.index)
y_bin

,astro-ph.CO,astro-ph.EP,astro-ph.GA,astro-ph.HE,astro-ph.IM,astro-ph.SR,cond-mat.dis-nn,cond-mat.mtrl-sci,cond-mat.soft,cond-mat.stat-mech,...,q-fin.PR,q-fin.RM,q-fin.ST,q-fin.TR,stat.AP,stat.CO,stat.ME,stat.ML,stat.OT,stat.TH
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51769,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
51770,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
51771,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
51772,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# getting the number of unique values in terms

unique_values = []

for i in range(len(paper_df)):
    for j in range(len(paper_df['terms'].iloc[i])):
        unique_values.append(paper_df['terms'].iloc[i][j])

unique_values = set(unique_values)
print(len(unique_values))

135


In [ ]:
X_details = paper_df.drop(['terms'], axis=1)

In [ ]:
# counting the total number of words in summaries
summaries = X_details['summaries']

count_words = summaries.str.findall(r'(\w+)').str.len()
print(count_words.sum())

9354795


Data Preprocessing

In [ ]:
#  I will preprocess the summaries


def preprocess_summary(text):
    """
    Cleans a pandas Series of text using:
    - Lowercasing
    - Stemming
    - Lemmatization
    - Stopword removal
    - Special character and short/long word removal
    """

    nltk.download("stopwords")
    nltk.download("wordnet")
    nltk.download("omw-1.4")

    stop_words = set(stopwords.words("english"))
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()

    # Lowercase
    text = text.str.lower()

    # Apply stemming - (to root word)
    text = text.apply(lambda x: " ".join([
        stemmer.stem(i)
        for i in re.sub("[^a-zA-Z]", " ", x).split()
        if i not in stop_words
    ]))

    # Apply lemmatization (to dict form)
    # text = text.apply(lambda x: " ".join([
    #     lemmatizer.lemmatize(i)
    #     for i in re.sub("[^a-zA-Z]", " ", x).split()
    #     if i not in stop_words
    # ]))

    # Remove non-letter characters followed by space
    text = text.apply(lambda x: re.sub(r"[^a-z]\s", "", x))

    # Remove hashtags
    text = text.str.replace("#", "", regex=False)

    # Remove words with len < 3 or > 7
    text = text.apply(lambda x: " ".join([
        w for w in x.split() if 2 < len(w) < 8
    ]))

    # Final stopword removal
    text = text.apply(lambda x: " ".join([
        word for word in x.split() if word not in stop_words
    ]))

    return text

summaries = preprocess_summary(summaries)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
summaries

,summaries
0,stereo match one wide use infer depth stereo i...
1,recent advanc combin extens amount data gener ...
2,paper propos novel mutual consist network net ...
3,consist train proven advanc semi achiev promis...
4,ensur safeti autom drive correct percept situa...
...,...
51769,address problem analyz set noisi time vari sig...
51770,discuss method track molecul global optim conn...
51771,attempt set foundat amino acid chain measur si...
51772,diffus tensor imag dti allow estim posit orien...


In [ ]:
count_words = summaries.str.findall(r'(\w+)').str.len()
print(count_words.sum())

4940220


In [ ]:
# removing any words that doesn't appear more than2
# removin rare words

from collections import Counter
from itertools import chain
import pandas as pd

def remove_rare_words(text_series: pd.Series, min_freq: int = 2):
    """
    Removes words that appear fewer than `min_freq` times across the entire corpus.

    """
    # Split each sentence into word list
    word_lists = text_series.str.split().tolist()

    # Flatten and count all word frequencies
    word_freq = Counter(chain.from_iterable(word_lists))

    # Remove words below frequency threshold and join back
    filtered_texts = [
        ' '.join([word for word in words if word_freq[word] >= min_freq])
        for words in word_lists
    ]

    return pd.Series(filtered_texts)

summaries = remove_rare_words(summaries)

In [ ]:
summaries

,0
0,stereo match one wide use infer depth stereo i...
1,recent advanc combin extens amount data gener ...
2,paper propos novel mutual consist network net ...
3,consist train proven advanc semi achiev promis...
4,ensur safeti autom drive correct percept situa...
...,...
51769,address problem analyz set noisi time vari sig...
51770,discuss method track molecul global optim conn...
51771,attempt set foundat amino acid chain measur si...
51772,diffus tensor imag dti allow estim posit orien...


In [ ]:
count_words = summaries.str.findall(r'(\w+)').str.len()
print(count_words.sum())

4933861


In [ ]:
# converting the X into array, so i can vectorize it
y = y_bin
X = np.array(summaries)

X.shape, y.shape

((51774,), (51774, 135))

In [ ]:
y

,astro-ph.CO,astro-ph.EP,astro-ph.GA,astro-ph.HE,astro-ph.IM,astro-ph.SR,cond-mat.dis-nn,cond-mat.mtrl-sci,cond-mat.soft,cond-mat.stat-mech,...,q-fin.PR,q-fin.RM,q-fin.ST,q-fin.TR,stat.AP,stat.CO,stat.ME,stat.ML,stat.OT,stat.TH
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51769,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
51770,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
51771,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
51772,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
y.shape

(51774, 135)

In [ ]:
# # removing the low frequency labels

# label_counts = y.sum(axis=0)
# y = y.loc[:, label_counts[label_counts >= 2].index]

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(
#         X, y, test_size=0.2, random_state= 42, shuffle=True, stratify=None
#     )

# X_test = list(map(''.join, np.array(X_test).tolist()))


In [ ]:
# from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
# import numpy as np

def split_and_vectorize(X, y, test_size=0.2, stop_words='english', max_df=0.5, min_df=0.01, random_state=42):
    """
    Splits text data into train/test sets and vectorizes it using TF-IDF.
    """
    # Safe non-stratified split for multi-label
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state, shuffle=True, stratify=None
    )

    # Ensure string format
    X_train = list(map(''.join, np.array(X_train).tolist()))
    X_test = list(map(''.join, np.array(X_test).tolist()))

    # TF-IDF vectorization
    vectorizer = TfidfVectorizer(stop_words=stop_words, max_df=max_df, min_df=min_df)
    X_train_tfidf = vectorizer.fit_transform(X_train)
    X_test_tfidf = vectorizer.transform(X_test)
    print(y_train.shape)
    # Preview vocabulary
    print("Sample TF-IDF feature names:", vectorizer.get_feature_names_out()[:10])

    return X_train_tfidf, X_test_tfidf, X_train, X_test, y_train, y_test, vectorizer

X_train_tfidf, X_test_tfidf, X_train, X_test, y_train, y_test, vectorizer = split_and_vectorize(X, y)

(41419, 135)
Sample TF-IDF feature names: ['abil' 'abl' 'ablat' 'acceler' 'access' 'accord' 'account' 'accur'
 'achiev' 'acquir']


In [ ]:
X_train_tfidf.shape, X_test_tfidf.shape, y_train.shape, y_test.shape

((41419, 914), (10355, 914), (41419, 135), (10355, 135))

In [ ]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.naive_bayes import ComplementNB
from sklearn.linear_model import LogisticRegression

def train_complement_nb(X_train_tfidf, y_train):
    """
    Trains a MultiOutputClassifier using Complement Naive Bayes.

    Args:
        X_train_tfidf: TF-IDF features for training
        y_train: Multi-label binary matrix

    Returns:
        trained model
    """
    model = MultiOutputClassifier(ComplementNB())
    model.fit(X_train_tfidf, y_train)
    print("Train accuracy (ComplementNB):", model.score(X_train_tfidf, y_train))
    print("Test accuracy (ComplementNB):", model.score(X_test_tfidf, y_test))
    return model

nb_model = train_complement_nb(X_train_tfidf, y_train)

Train accuracy (ComplementNB): 0.03541852772882011
Test accuracy (ComplementNB): 0.03582810236600676


In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
def train_logistic_regression(X_train_tfidf, y_train):
    """
    Trains a MultiOutputClassifier using Logistic Regression.

    Args:
        X_train_tfidf: TF-IDF features for training
        y_train: Multi-label binary matrix (with no all-zero columns)

    Returns:
        trained model
    """
    # Filter out all-zero columns
    valid_labels = y_train.columns[y_train.sum(axis=0) > 0]
    y_train_filtered = y_train[valid_labels]

    model = MultiOutputClassifier(LogisticRegression(max_iter=1000))
    model.fit(X_train_tfidf, y_train_filtered)
    print("Train accuracy (LogisticRegression):", model.score(X_train_tfidf, y_train_filtered))
    print("Test accuracy (LogisticRegression):", model.score(X_test_tfidf, y_test[valid_labels]))
    return model, valid_labels

log_model, valid_labels = train_logistic_regression(X_train_tfidf, y_train)

Train accuracy (LogisticRegression): 0.43400371810038874
Test accuracy (LogisticRegression): 0.4198937711250604


In [ ]:
y_train.shape

(41419, 135)

In [ ]:

# Predict with Naive Bayes
y_pred_nb = nb_model.predict(X_test_tfidf)
predicted_terms_nb = mlb.inverse_transform(y_pred_nb)

# Predict with Logistic Regression
mlb_lr = MultiLabelBinarizer(classes=valid_labels)
mlb_lr.fit([])
y_pred_lr = log_model.predict(X_test_tfidf)
predicted_terms_lr = mlb_lr.inverse_transform(y_pred_lr)


In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred_nb, target_names=y_test.columns))


                    precision    recall  f1-score   support

       astro-ph.CO       0.00      0.00      0.00         0
       astro-ph.EP       0.00      0.00      0.00         1
       astro-ph.GA       0.00      0.00      0.00         0
       astro-ph.HE       0.00      0.00      0.00         0
       astro-ph.IM       0.00      0.33      0.01         6
       astro-ph.SR       0.00      0.00      0.00         1
   cond-mat.dis-nn       0.01      0.91      0.02        11
 cond-mat.mtrl-sci       0.01      0.38      0.01         8
     cond-mat.soft       0.00      1.00      0.01         1
cond-mat.stat-mech       0.01      0.80      0.02        10
   cond-mat.str-el       0.00      0.00      0.00         1
             cs.AI       0.31      0.66      0.42      1653
             cs.AR       0.01      0.86      0.02         7
             cs.CC       0.00      0.29      0.01         7
             cs.CE       0.00      0.22      0.01        18
             cs.CG       0.02      0.79

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
y_test_filtered = y_test[valid_labels]


print(classification_report(
    y_test_filtered,
    y_pred_lr,
    target_names=valid_labels,
    zero_division=0
))

                    precision    recall  f1-score   support

       astro-ph.CO       0.00      0.00      0.00         0
       astro-ph.GA       0.00      0.00      0.00         0
       astro-ph.HE       0.00      0.00      0.00         0
       astro-ph.IM       0.00      0.00      0.00         6
       astro-ph.SR       0.00      0.00      0.00         1
   cond-mat.dis-nn       0.00      0.00      0.00        11
 cond-mat.mtrl-sci       0.00      0.00      0.00         8
     cond-mat.soft       0.00      0.00      0.00         1
cond-mat.stat-mech       0.00      0.00      0.00        10
   cond-mat.str-el       0.00      0.00      0.00         1
             cs.AI       0.60      0.16      0.25      1653
             cs.AR       0.00      0.00      0.00         7
             cs.CC       0.00      0.00      0.00         7
             cs.CE       0.00      0.00      0.00        18
             cs.CG       0.00      0.00      0.00        28
             cs.CL       0.56      0.22

In [ ]:
# X_test

In [ ]:
comparison_df = pd.DataFrame({
    'summary': X_test,
    'nb_predicted_terms': predicted_terms_nb,
    'lr_predicted_terms': predicted_terms_lr
})

comparison_df.head()

,summary,nb_predicted_terms,lr_predicted_terms
0,depth predict play crucial role scene recent m...,"(cs.CV, cs.GR, cs.RO)","(cs.CV,)"
1,deep gener model made tremend advanc imag sign...,"(cs.AI, cs.CG, cs.CR, cs.GR, cs.IT, cs.LG, cs....","(cs.LG, stat.ML)"
2,propos novel point cloud segment name jointli ...,"(cs.CV,)","(cs.CV,)"
3,propos revisit train object target class weakl...,"(cs.CV, eess.IV)","(cs.CV,)"
4,heart failur major compon lead caus mortal des...,"(cs.AR, cs.CE, cs.CY, cs.LG, cs.NA, cs.NE, cs....","(cs.CV, cs.LG)"


In [ ]:
def predict_topics(summary: str, model, vectorizer, valid_labels):
    """
    Predict topics for a single summary using a trained model.

    Parameters:
    - summary: str, the input paper summary
    - model: trained classification model (e.g., Naive Bayes or Logistic Regression)
    - vectorizer: trained TF-IDF vectorizer
    - mlb: fitted MultiLabelBinarizer

    Returns:
    - List of predicted topics
    """
    # Preprocess summary: basic clean-up (adjust to your pipeline if needed)
    # import re
    # from nltk.corpus import stopwords
    # from nltk.stem import PorterStemmer
    # import nltk

    nltk.download("stopwords", quiet=True)
    stop_words = set(stopwords.words("english"))
    stemmer = PorterStemmer()

    mlb_lr = MultiLabelBinarizer(classes=valid_labels)
    mlb_lr.fit([])
    # Lowercase and clean
    summary = summary.lower()
    summary = re.sub(r"[^a-zA-Z]", " ", summary)
    summary = " ".join([stemmer.stem(w) for w in summary.split() if w not in stop_words and 2 < len(w) < 8])

    # Transform with TF-IDF
    summary_tfidf = vectorizer.transform([summary])

    # Predict and decode
    y_pred = model.predict(summary_tfidf)
    predicted_topics = mlb_lr.inverse_transform(y_pred)

    return predicted_topics[0]  # Return first (and only) prediction


In [ ]:
single_summary = "We present a new method for quantum entanglement detection using graph neural networks."

predicted_topics = predict_topics(
    summary=single_summary,
    model=log_model,               # or nb_model
    vectorizer=vectorizer,
    valid_labels=valid_labels                        # make sure mlb is fit with correct classes
)

print("Predicted topics:", predicted_topics)


Predicted topics: ('cs.LG', 'stat.ML')
